In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
os.chdir('dataset/')

In [3]:
import glob
glob.glob('*.csv')

['application_record.csv', 'cleaned_df.csv', 'credit_record.csv']

In [4]:
final_df = pd.read_csv('cleaned_df.csv')
final_df.head()

,Unnamed: 0,name_income_type,order_name_education_type,id,cnt_children,amt_income_total,name_family_status,name_housing_type,days_birth,days_employed,cnt_fam_members,months_balance,code_gender_F,code_gender_M,flag_own_realty_N,flag_own_realty_Y,occupation_type_code,response
0,0,1,1,5010848,0,54000.0,3,6,61.03,0.0,2.0,0,False,True,False,True,9,1.0
1,1,1,1,5010848,0,54000.0,3,6,61.03,0.0,2.0,1,False,True,False,True,9,1.0
2,2,1,1,5010848,0,54000.0,3,6,61.03,0.0,2.0,2,False,True,False,True,9,1.0
3,3,1,1,5010848,0,54000.0,3,6,61.03,0.0,2.0,3,False,True,False,True,9,1.0
4,4,1,1,5010848,0,54000.0,3,6,61.03,0.0,2.0,4,False,True,False,True,9,1.0


In [5]:
final_df.columns

Index(['Unnamed: 0', 'name_income_type', 'order_name_education_type', 'id',
       'cnt_children', 'amt_income_total', 'name_family_status',
       'name_housing_type', 'days_birth', 'days_employed', 'cnt_fam_members',
       'months_balance', 'code_gender_F', 'code_gender_M', 'flag_own_realty_N',
       'flag_own_realty_Y', 'occupation_type_code', 'response'],
      dtype='object')

In [6]:
final_df.code_gender_F.dtype

dtype('bool')

In [7]:
def lable(x):
    if x == True:
        return 1
    elif x == False:
        return 0

for cols in ['code_gender_F', 'code_gender_M', 'flag_own_realty_N','flag_own_realty_Y']:
    final_df[cols] = final_df[cols].apply(lable)

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import CategoricalNB
from xgboost import XGBClassifier

In [10]:
rf  = RandomForestClassifier()
lr = LogisticRegression()
nb = CategoricalNB()
xgb = XGBClassifier()
model_list  = [rf, lr,nb,xgb]
model_name = ['rf','lr','nb','xgb']
model_output  = {}
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
x = final_df.iloc[:,:-1]
y = final_df.iloc[:,-1]
train_x, test_x, train_y, test_y = train_test_split(x,y, test_size=0.3,
                                                   random_state=49,
                                                   stratify=y)

In [11]:
model_dfs = {}
for i, j in zip(model_list, model_name):
    model_dfs[j] = i.fit(train_x, train_y)

In [12]:
model_dfs.keys()

dict_keys(['rf', 'lr', 'nb', 'xgb'])

In [13]:
test_pred = []
final_pred  = np.zeros((test_x.shape[0],))
for i in model_dfs.keys():
    test_pred.append(model_dfs[i].predict(test_x))
    final_pred +=model_dfs[i].predict(test_x)

In [16]:
for i in model_dfs.keys():
    pred  = model_dfs[i].predict(test_x)
    print('Model Name %s'%i)
    print(classification_report(test_y, pred))

Model Name rf
              precision    recall  f1-score   support

         0.0       0.50      0.43      0.46      3473
         1.0       0.99      0.99      0.99    229842

    accuracy                           0.99    233315
   macro avg       0.75      0.71      0.73    233315
weighted avg       0.98      0.99      0.98    233315

Model Name lr
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      3473
         1.0       0.99      1.00      0.99    229842

    accuracy                           0.99    233315
   macro avg       0.49      0.50      0.50    233315
weighted avg       0.97      0.99      0.98    233315

Model Name nb
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      3473
         1.0       0.99      1.00      0.99    229842

    accuracy                           0.99    233315
   macro avg       0.49      0.50      0.50    233315
weighted avg       0.97      0.9

In [17]:
for i in model_dfs.keys():
    pred  = model_dfs[i].predict(test_x)
    print('Model Name %s'%i)
    print('pos_label : 1  ::',fbeta_score(test_y, pred,beta=0.5,pos_label=1))

Model Name rf
pos_label : 1  :: 0.9918394468717157
Model Name lr
pos_label : 1  :: 0.9880560776268977
Model Name nb
pos_label : 1  :: 0.9880560776268977
Model Name xgb
pos_label : 1  :: 0.9891298544044674
